# Using XGBoost to Forecast NSW Energy Demand
This notebook will be used for developing an XGBoost model that will be used for forecasting the demand in NSW, with the particular goal of looking at how residential (small scale) solar PV impacts the daily maximum, minimum and range of demand in NSW.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
import multiprocessing

In [2]:
# set style for plots
sns.set_theme()

In [3]:
# read in data
df = pd.read_csv('../data/raw/all_data.csv')
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df.sort_index()
df.head()

,demand_min,demand_max,temp_min,temp_max,temp_mean,units,cum_units,output,cum_output,population
datetime,,,,,,,,,,
2010-01-01,6157.36,8922.42,22.1,28.8,25.094000,1790.000000,21518.000000,2767.019000,30362.926000,7.144292e+06
2010-01-02,6112.73,9326.64,21.6,29.4,24.765385,1810.677419,21596.419355,2811.171258,30496.336935,7.144495e+06
2010-01-03,6014.91,8277.85,17.9,21.5,19.429825,1831.354839,21674.838710,2855.323516,30629.747871,7.144699e+06
2010-01-04,6023.79,9522.30,17.9,23.9,20.625926,1852.032258,21753.258065,2899.475774,30763.158806,7.144902e+06
2010-01-05,6287.12,10728.72,15.4,27.7,22.660417,1872.709677,21831.677419,2943.628032,30896.569742,7.145106e+06


In [4]:
def create_date_predictors(dataframe):
    df = dataframe.copy()
    df['dayofweek'] = df.index.dayofweek
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    return df

In [5]:
# Add in datetime predictors
df = create_date_predictors(df)

In [6]:
df.head()

,demand_min,demand_max,temp_min,temp_max,temp_mean,units,cum_units,output,cum_output,population,dayofweek,month,year,dayofyear,dayofmonth
datetime,,,,,,,,,,,,,,,
2010-01-01,6157.36,8922.42,22.1,28.8,25.094000,1790.000000,21518.000000,2767.019000,30362.926000,7.144292e+06,4,1,2010,1,1
2010-01-02,6112.73,9326.64,21.6,29.4,24.765385,1810.677419,21596.419355,2811.171258,30496.336935,7.144495e+06,5,1,2010,2,2
2010-01-03,6014.91,8277.85,17.9,21.5,19.429825,1831.354839,21674.838710,2855.323516,30629.747871,7.144699e+06,6,1,2010,3,3
2010-01-04,6023.79,9522.30,17.9,23.9,20.625926,1852.032258,21753.258065,2899.475774,30763.158806,7.144902e+06,0,1,2010,4,4
2010-01-05,6287.12,10728.72,15.4,27.7,22.660417,1872.709677,21831.677419,2943.628032,30896.569742,7.145106e+06,1,1,2010,5,5


In [7]:
# Create 5, 1 year splits
splits = TimeSeriesSplit(n_splits=5, test_size=365)
# set up data for model
y_min = df['demand_min']
y_max = df['demand_max']
X = df.drop(['demand_max', 'demand_min'], axis=1)

In [19]:
# set up model
search_params = {'max_depth': [2, 4, 6, 8], 'n_estimators':[100, 250, 500, 750, 1000], 'learning_rate': [0.01, 0.05, 0.1]}
xgb_model = xgb.XGBRegressor(n_jobs=multiprocessing.cpu_count()//2)
clf_max = GridSearchCV(estimator=xgb_model,
                       param_grid=search_params,
                       cv=splits,
                       n_jobs=2,
                       verbose=True)

In [20]:
clf_max.fit(X, y_max, verbose=100)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=365),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, impo...
                                    max_cat_threshold=None,
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=8, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=2,
             param_grid={'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [2, 4, 6, 8],
                         'n_estimators': [100, 250, 500, 750, 1000]},
             verbose=True)

In [21]:
clf_max.best_params_

{'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 100}